In [1]:
import numpy as np
import pandas as pd
import codecs
import geocoder
import folium
from folium.plugins import HeatMap

In [2]:
elesch = pd.read_csv('sch_data/elesch_fuji_name.csv', encoding='utf-16', names=['name'])
elesch['address'] = pd.read_csv('sch_data/elesch_fuji_address.csv', encoding='utf-16', names=['address'])

In [3]:
juhisch = pd.read_csv('sch_data/juhisch_fuji_name.csv', encoding='utf-16', names=['name'])
juhisch['address'] = pd.read_csv('sch_data/juhisch_fuji_address.csv', encoding='utf-16', names=['address'])

In [4]:
def geocoding(a): # geocoding
    Localname = str(a) #u'{}'.format(str(a))
    g = geocoder.google(Localname)
    return g.latlng

In [ ]:
geocoding('東京都足立区南花畑1-3-6')

In [ ]:
def mk_latlng_col(df, address_col): # Add lat, lng columns to df.
    latlng_list = []
    for index, row in df.iterrows():
        latlng = geocoding(row[address_col])
        while latlng == None or latlng == 0:
            latlng = geocoding(row[address_col])
        print(latlng)
        latlng_list.append(latlng)
#     print(latlng_list)
    latlng_df = pd.DataFrame(np.array(latlng_list), columns=['lat', 'lng'])
    return pd.concat([df, latlng_df], axis=1)
    #     print(type(row[1]))
    #     print(index, geocoding(str(row[1])))
    # elesch['lat'] = 

In [ ]:
elesch_loc = mk_latlng_col(elesch, 1)
juhisch_loc = mk_latlng_col(juhisch, 1)

In [ ]:
# Make df of elesch and juhisch
ele_juhi_sch = pd.concat([elesch_loc, juhisch_loc], ignore_index=True, axis=0)

In [ ]:
ele_juhi_sch

## Heatmap of school

In [ ]:
def mk_heatmap(df, hm=None): # Make a heatmap
    fujisawa = [35.37,139.47]
    center = fujisawa
    hmap = folium.Map(location=center, zoom_start=12)
    marker_list = []
    for i in range(len(df.index)):
        if hm == None:
            marker_latlng = [df.at[df.index.values[i],'lat'], df.at[df.index.values[i],'lng']]
            input_list = marker_latlng
        else:
            marker_latlngx = [df.at[df.index.values[i],'lat'], df.at[df.index.values[i],'lng'], df.loc[df.index.values[i], hm]]
            input_list = marker_latlngx
        marker_list.append(input_list)
    if hm == None:
        hmarker = HeatMap(marker_list)
    else:
        hmarker = HeatMap(marker_list, max_zoom=13, radius=40, blur=18)
    hmap.add_child(hmarker)
    return hmap

In [ ]:
mk_heatmap(ele_juhi_sch)

In [ ]:
mk_heatmap(elesch_loc)

In [ ]:
mk_heatmap(juhisch_loc)